In [2]:
import os
import pandas as pd
import yfinance as yf

'''自訂函數'''


# 取得資料：本地或是 Yahoo
def getDataFM(prod, st, en):
    # 檔案路徑
    bakfile = f"data/YF_{prod}_{st}_{en}_stock_daily_adj.csv"
    # 假如已經有 Excel 資料檔案
    if os.path.exists(bakfile):
        # 讀取 Excel 資料檔案
        data = pd.read_csv(bakfile, index_col="Date", parse_dates=True)
    # 假如 Excel 資料檔案不存在
    else:
        # 透過 yf 下載
        data = yf.download(f"{prod}.TW", start=st, end=en)
        # 下載的資料是英文的，處理欄位名稱，轉換為小寫
        # 轉換部分不包含索引，所以日期還是大寫開頭的 Date
        data.columns = [i.lower() for i in data.columns]
        # 假如下載後沒有資料
        if data.empty:
            print("無法從 Yahoo Finance 下載資料")
            # 傳回一個空的 pd
            return pd.DataFrame()
        # 假如資料 data 不是空的，依照指定路徑存擋
        data.to_csv(bakfile)
    # 存檔後，傳回資料
    return data

# 取得回測資料
prod = "0050"
data = getDataFM(prod, "2013-01-01", "2023-12-15")

# 初始部位
position = 0
trade = pd.DataFrame()
# 開始回測
for i in range(data.shape[0] - 1):
    # 取得策略會應用到的變數
    c_time = data.index[i]
    c_low = data.loc[c_time, "low"]
    c_high = data.loc[c_time, "high"]
    c_close = data.loc[c_time, "close"]
    c_open = data.loc[c_time, "open"]
    # 取下一期資料做為進場資料
    n_time = data.index[i + 1]
    n_open = data.loc[n_time, "open"]

    # 進場程序
    if position == 0:
        # 進場邏輯
        if c_close > c_open and (c_close - c_open) * 2 < (c_open - c_low):
            position = 1
            order_i = i
            order_time = n_time
            order_price = n_open
            order_unit = 1
    # 出場程序
    elif position == 1:
        # 出場邏輯
        if i > order_i + 3 and c_close > c_open:
            position = 0
            cover_time = n_time
            cover_price = n_open
            # 交易紀錄
            trade = trade._append(
                pd.Series(
                    [
                        prod,
                        "Buy",
                        order_time,
                        order_price,
                        cover_time,
                        cover_price,
                        order_unit,
                    ]
                ),
                ignore_index=True,
            )
# 顯示交易紀錄
print(trade)

        0    1          2           3          4           5  6
0    0050  Buy 2009-01-06   36.590000 2009-01-13   34.189999  1
1    0050  Buy 2009-02-02   30.400000 2009-02-09   29.950001  1
2    0050  Buy 2009-02-27   30.510000 2009-03-05   35.040001  1
3    0050  Buy 2009-04-08   40.000000 2009-04-14   42.990002  1
4    0050  Buy 2009-06-17   43.049999 2009-06-23   42.849998  1
..    ...  ...        ...         ...        ...         ... ..
177  0050  Buy 2021-10-25  136.000000 2021-11-02  136.800003  1
178  0050  Buy 2021-11-12  139.899994 2021-11-18  141.750000  1
179  0050  Buy 2021-12-08  143.100006 2021-12-16  141.350006  1
180  0050  Buy 2022-01-13  149.550003 2022-01-21  143.899994  1
181  0050  Buy 2022-02-14  143.949997 2022-02-18  143.500000  1

[182 rows x 7 columns]
